This notebook will produce a graph as follows:
- X axis is
    - em-var status (categorical)
    - mean predicted activity bin
- Y-axis is the odds ratio of phylop significance representation in that category

In [1]:
import glob
import pandas as pd

In [2]:
#importlib code just for debugging : to allow changes in scripts we are imporing from
#reflect here. It can be removed if you are not making changes to the scripts we are imporing from

import importlib ##can remove
import graphing_config as gc
import or_common as orc
importlib.reload(gc) ##can remove
importlib.reload(orc) ##can remove

<module 'or_common' from '/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_scripts/5.graphs/odds_ratio/or_common.py'>

In [3]:
#phylop_emvar=pd.read_csv(glob.glob(GC.data_base_path+"phylop_emvar/*.csv")[0])
master_table=pd.read_csv(glob.glob("/home/mcn26/varef/scripts/noon_data/4.old.count/count_all.csv/*.csv")[0])

In [4]:
master_table

,CADD>=10,CADD>=20,CADD>=30,CADD>=40,CADD>=50,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,...,SKNSH__skew_(1&5^ Inf),SKNSH__ref_(-Inf^-1),SKNSH__ref_[-1^0),SKNSH__ref_[0^1),SKNSH__ref_[1^2),SKNSH__ref_[2^3),SKNSH__ref_[3^4),SKNSH__ref_[4^5),SKNSH__ref_[5^Inf),count
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,84
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,2123
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,149
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,578
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,1930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360364,False,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,1
3360365,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3360366,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,1
3360367,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,1


In [5]:
#extract some relevant sets of columns & drop all the columns we don't need...

columns=master_table.columns.to_list()

genomic_regions_cols=[i for i in columns if i.startswith("is_in_") ]
mean_activity_cols=[i for i in columns if i.startswith("mean_skew_")]
emVar_cols=[i for i in columns if i.startswith("emVar_")]

columns_to_keep=genomic_regions_cols+mean_activity_cols+emVar_cols+["count","phylop_significant"]


master_table=master_table[columns_to_keep]

In [6]:
#remove rows we don't need. Rows which are identical except for their counts
#were distinct rows because of columns we have discarded above
#so this summing finishes the above subsetting. 
master_table=master_table.groupby(["phylop_significant"]+emVar_cols+mean_activity_cols+genomic_regions_cols).sum()
master_table.reset_index(inplace=True)

In [20]:
#fix the dumb character substitutions
#(pyspark hates certain characters, so we had to put substitutes)
#both in the actual table...
master_table.columns=[col_name.replace("&", ".").replace("^",",") for col_name in master_table.columns]
#...and in the column names we extracted above. 
mean_activity_cols=[i.replace("&", ".").replace("^",",") for i in mean_activity_cols]

In [43]:
#select one combo of subset cata
selected=master_table[master_table["is_in_PLS"]]
#drop now irrelevant cata columns
selected=selected.drop(genomic_regions_cols,axis=1)
#break into multiple tables
dic={k:selected[selected[k]] for k in mean_activity_cols}

mean_activity_strata_collector={}
for mean_activity_strata in dic.keys():
    
    emvar_collector={}
    for emVar in emVar_cols:
    
        emvar_collector[emVar]=orc.compute_OR(df=dic[mean_activity_strata],
                      a=emVar,
                      a_val=True,
                      b="phylop_significant",
                      b_val=True)
        
        #break
    mean_activity_strata_collector[mean_activity_strata]=emvar_collector



/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:156: RuntimeWarning: divide by zero encountered in divide
  se = np.sqrt((1/table).sum())
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:168: RuntimeWarning: invalid value encountered in scalar subtract
  loglow = log_or - z*se
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:155: RuntimeWarning: divide by zero encountered in log
  log_or = np.log(oddsratio)
/home/mcn26/.conda/envs/mcn_varef/lib/python3.10/site-packages/scipy/stats/_odds_ratio.py:169: RuntimeWarning: invalid value encountered in scalar add
  loghigh = log_or + z*se


In [47]:

mean_activity_strata_collector

{'mean_skew_(-Inf, -1.5)': {'emVar_K562': {'OR': 4.5490331491712706,
   'ci_lower': 1.5587538029879107,
   'ci_upper': 13.275799265151544,
   'p': 0.0021056535182230524},
  'emVar_SKNSH': {'OR': inf, 'ci_lower': nan, 'ci_upper': inf, 'p': 1.0},
  'emVar_HepG2': {'OR': nan, 'ci_lower': 0, 'ci_upper': inf, 'p': 1.0}},
 'mean_skew_[-1.5, -1.0)': {'emVar_K562': {'OR': 2.947866584480515,
   'ci_lower': 1.9849143064944053,
   'ci_upper': 4.37798114077995,
   'p': 5.508470011452358e-09},
  'emVar_SKNSH': {'OR': 2.1967073516889015,
   'ci_lower': 1.5805536948847716,
   'ci_upper': 3.053058687332901,
   'p': 1.1915607358557522e-06},
  'emVar_HepG2': {'OR': 1.8004852556924225,
   'ci_lower': 1.0392734373581385,
   'ci_upper': 3.1192437326276883,
   'p': 0.03866177849090434}},
 'mean_skew_[-1.0, -0.5)': {'emVar_K562': {'OR': 1.4141668648696148,
   'ci_lower': 1.3393507885893547,
   'ci_upper': 1.4931621638879815,
   'p': 4.669224205546097e-37},
  'emVar_SKNSH': {'OR': 1.2655408261810006,
   'ci_l